In [1]:
%cd ..

/playpen-raid1/lin.tian/projects/pyReg/sotas/voxelmorph/scripts


In [7]:
# import progressive_train
import sys
sys.path.append("/playpen-raid2/lin.tian/projects/icon_lung/ICON_lung")
import train_for_gradICON as progressive_train
import voxelmorph as vxm
import torch
import numpy as np
from tools.evaluate_dir_lab import eval_copd_highres
import os
import torch.nn.functional as F

In [51]:
gpu_id = 3
weights_path = "/playpen-raid2/lin.tian/projects/icon_lung/ICON_lung_results/icon/comparing/vm_diff/2022_05_04_18_04_17/checkpoints/0080.pt"
root = "/playpen-raid1/lin.tian/data/lung/dirlab_highres_350"
cases = [f"copd{i}_highres" for i in range(1,11)]
hu_clip_range = [-1000, 0]

torch.cuda.set_device(gpu_id)

plot_root = f"{'/'.join(weights_path.split('/')[:-2])}/test"
if not os.path.exists(plot_root):
    os.mkdir(plot_root)

In [4]:
import matplotlib.pyplot as plt
def show(tensor):
    plt.imshow(tensor.cpu().detach())
    plt.colorbar()
    plt.show()

def save_to_itk(img, p):
    img_itk = itk.image_from_array(img)
    itk.imwrite(img_itk, p)

## Load trained net

In [52]:
# net = progressive_train.build_framework(progressive_train.make_2x_net(progressive_train.make_4x_net(progressive_train.make_8x_net())), [5, 1, 175, 175, 175])
# net = progressive_train.build_framework(progressive_train.make_4x_net(progressive_train.make_8x_net()), [6, 1, 87, 87, 87])
# net = progressive_train.build_framework(progressive_train.make_8x_net(), [24, 1, 43, 43, 43])

net = vxm.networks.VxmDense.load(weights_path, 'cuda')
net.cuda()
0

0

## Load data

In [10]:
# Load data
def process(iA, isSeg=False):
    iA = iA[None, None, :, :, :]
    if isSeg:
        iA = iA.float()
        iA = torch.nn.functional.max_pool3d(iA, 2)
        iA[iA>0] = 1
    else:
        iA = torch.clip(iA, hu_clip_range[0], hu_clip_range[1])
        iA = iA / 1000
        iA += 1

        iA = torch.nn.functional.avg_pool3d(iA, 2)
    return iA

def read_itk(path):
    img = itk.imread(path)
    return torch.tensor(np.asarray(img)), np.flipud(list(img.GetOrigin()))

import itk
import tqdm

dirlab = []
dirlab_seg = []
dirlab_origin = []
for name in tqdm.tqdm(list(iter(cases))[:]):
    image_insp, _ = read_itk(f"{root}/{name}/{name}_INSP_STD_COPD_img.nii.gz")
    image_exp, _ = read_itk(f"{root}/{name}/{name}_EXP_STD_COPD_img.nii.gz")
    seg_insp, _ = read_itk(f"{root}/{name}/{name}_INSP_STD_COPD_label.nii.gz")
    seg_exp, origin = read_itk(f"{root}/{name}/{name}_EXP_STD_COPD_label.nii.gz")

    dirlab.append((process(image_insp)*process(seg_insp, True), process(image_exp)*process(seg_exp, True)))

    # dirlab.append((process(image_insp), process(image_exp)))
    dirlab_seg.append((process(seg_insp, True), process(seg_exp, True)))
    dirlab_origin.append(origin)



100%|██████████| 10/10 [00:34<00:00,  3.49s/it]


In [11]:
dirlab_loader = torch.utils.data.DataLoader(
                        torch.utils.data.TensorDataset(
                            torch.stack(
                                [torch.cat(d, 0) for d in dirlab],
                                0
                            ),
                            torch.stack(
                                [torch.cat(d, 0) for d in dirlab_seg],
                                0
                            ),
                            torch.arange(0,10)
                        ),
                        batch_size = 1,
                        shuffle = False,
                        drop_last = True
                    )

## Inference on data

In [53]:

def normalize(phi, shape):
    for i in range(len(shape)):
            phi[:, i, ...] = 2 * (phi[:, i, ...] / (shape[i] - 1) - 0.5)
    return phi

phis = []
phis_inv = []

for img, seg, i in dirlab_loader:
    with torch.no_grad():
        (warped_A, _, phi_X_Y) = net(img[:,0].cuda(), img[:,1].cuda())
        (Warped_B, _, phi_Y_X) = net(img[:,1].cuda(), img[:,0].cuda())
        # net(img[:,0].cuda()+1, img[:,1].cuda()+1, seg[:,0].cuda(), seg[:,1].cuda())   
        phis.append(normalize(phi_X_Y+net.transformer.grid, [175, 175, 175]).detach())
        phis_inv.append(normalize(phi_Y_X+net.transformer.grid, [175, 175, 175]).detach())
        del phi_X_Y, phi_Y_X
    
    case_id = cases[i]
    if not os.path.exists(f"{plot_root}/{case_id}"):
        os.mkdir(f"{plot_root}/{case_id}")
    # save_to_itk(img[:,0].cpu().numpy()[0,0], f"{plot_root}/{case_id}/{case_id}_INSP.nii.gz")
    # save_to_itk(img[:,1].cpu().numpy()[0,0], f"{plot_root}/{case_id}/{case_id}_EXP.nii.gz")
    # save_to_itk(net.warped_image_A.detach().cpu().numpy()[0,0], f"{plot_root}/{case_id}/{case_id}_warped_INSP.nii.gz")
    # save_to_itk(net.warped_image_B.detach().cpu().numpy()[0,0], f"{plot_root}/{case_id}/{case_id}_warped_EXP.nii.gz")


In [54]:

phis_np = (torch.cat(phis).cpu().numpy() + 1.)/2.
phis_inv_np = (torch.cat(phis_inv).cpu().numpy() + 1.)/2.
res = eval_copd_highres("/playpen-raid1/lin.tian/data/lung/reg_lung_2d_3d_1000_dataset_4_proj_clean_bg/landmarks", 
                    root,
                    [c[:-8] for c in cases], 
                    phis_np, phis_inv_np,  
                    np.repeat(np.array([[1.,1.,1.]]), len(cases), axis=0), 
                    np.array([np.flipud(i) for i in dirlab_origin]),
                    np.repeat(np.array([[350,350,350]]), len(cases), axis=0))

In [55]:
results = []
results_inv = []
for k,v in res.items():
    result = []
    for m,n in v.items():
        if m in ["mTRE", "mTRE_X", "mTRE_Y", "mTRE_Z", "dice", "folds"]:
            result.append(n)
    if '_inv' in k:
        results_inv.append(result)
    else:
        results.append(result)
results = np.array(results)
print(f"phi:exp->insp: mTRE: {results[:,0].mean()}," \
        f"mTRE_X: {results[:,1].mean()}, mTRE_Y: {results[:,2].mean()}, "\
        f"mTRE_Z: {results[:,3].mean()}, DICE: {results[:,4].mean()},"\
        f"Folds: {results[:,5].mean()}")

results_inv = np.array(results_inv)
print(f"phi:insp->exp: mTRE: {results_inv[:,0].mean()}, "\
        f"mTRE_X: {results_inv[:,1].mean()}, mTRE_Y: {results_inv[:,2].mean()}, "\
        f"mTRE_Z: {results_inv[:,3].mean()}, DICE: {results_inv[:,4].mean()},"\
        f"Folds: {results_inv[:,5].mean()}")

phi:exp->insp: mTRE: 23.77874442407013,mTRE_X: 4.1071797285104195, mTRE_Y: 15.715032932250347, mTRE_Z: 13.95094977376375, DICE: 0.9696588573680909,Folds: 0.060322428867220876
phi:insp->exp: mTRE: 19.291513302516414, mTRE_X: 4.040525158548921, mTRE_Y: 12.941997275634893, mTRE_Z: 10.59692605698076, DICE: 0.9766348349030023,Folds: 0.17269663140177727


phi:exp->insp: mTRE: 5.302871935245742,mTRE_X: 1.777886032561937, mTRE_Y: 3.200885882510536, mTRE_Z: 2.790754698866906, DICE: 0.9700514042800359,Folds: 0.006547046732157469
phi:insp->exp: mTRE: 4.757475510573636, mTRE_X: 1.5857615342378473, mTRE_Y: 3.1099422082680417, mTRE_Z: 2.3062457764745705, DICE: 0.9771946064865574,Folds: 0.01225886596366763

phi:exp->insp: mTRE: 4.001431166553917,mTRE_X: 1.324181510158429, mTRE_Y: 2.432875768733552, mTRE_Z: 2.1077003890919555, DICE: 0.9727557430115337,Folds: 0.0
phi:insp->exp: mTRE: 3.075129913806052, mTRE_X: 1.1066196591534747, mTRE_Y: 1.8337419005307845, mTRE_Z: 1.591581931360544, DICE: 0.9770073735674496,Folds: 0.0

In [56]:
results

array([[26.74989545,  4.51868155, 24.44018878,  7.73409789,  0.976795  ,
         0.06558436],
       [22.32693825,  3.37044283, 20.20433471,  6.47178029,  0.972007  ,
         0.04538704],
       [13.03711165,  2.26997802,  6.66917142, 10.11066804,  0.97888778,
         0.0577636 ],
       [30.04133659,  5.4338951 , 17.43349508, 20.55213587,  0.94850213,
         0.05605518],
       [30.59479548,  4.47166506, 24.17676673, 15.55952323,  0.95941916,
         0.07439222],
       [28.70293266,  3.55733975, 24.7809661 , 11.93693581,  0.96879972,
         0.05546672],
       [22.13528887,  3.45896367,  9.81759824, 18.50217758,  0.97235305,
         0.0643505 ],
       [27.08143533,  3.93466719, 14.51734165, 20.15877724,  0.96881454,
         0.05964286],
       [15.16325283,  3.91837572,  7.86001413, 10.23840781,  0.9732977 ,
         0.03935062],
       [21.95445714,  6.13778838,  7.25045248, 18.24499398,  0.9777125 ,
         0.0852312 ]])

In [57]:
def dist(l1, l2):
    return np.sqrt(np.sum((l1-l2)**2, 1))

In [58]:
perc = []
perc_inv = []
for i in range(len(cases)):
    case_id = f"{cases[i][:-8]}"
    d = dist(res[case_id]["target_list"], res[case_id]["warped_list"])
    perc.append(np.percentile(d, (25, 50, 75)))

    case_id = f"{cases[i][:-8]}_inv"
    d = dist(res[case_id]["target_list"], res[case_id]["warped_list"])
    perc_inv.append(np.percentile(d, (25, 50, 75)))

print(f"phi:exp->insp: percentile: {np.array(perc).mean(0)}")
print(f"phi:insp->exp: percentile: {np.array(perc_inv).mean(0)}")

phi:exp->insp: percentile: [16.17361173 22.92352632 30.46026184]
phi:insp->exp: percentile: [12.92725719 18.68890011 25.0141347 ]


## Save landmarks

In [59]:
def save_landmarks(landmarks, values, shape, save_path):
    img = np.zeros(shape)
    for i in range(landmarks.shape[0]):
        p = [int(x) for x in landmarks[i]]
        img[p[2]:p[2]+2, p[1]:p[1]+2, p[0]:p[0]+2] = values[i]
    save_to_itk(img, save_path)



In [60]:
# Save result into landmarks
for k,v in res.items():
    if "_inv" in k:
        if not os.path.exists(f"{plot_root}/{k[:-4]}_highres"):
            os.mkdir(f"{plot_root}/{k[:-4]}_highres")
        save_landmarks(v["target_list"]/np.array((2,2,2)), np.array(range(v["target_list"].shape[0])), (175, 175, 175), f"{plot_root}/{k[:-4]}_highres/{k[:-4]}_highres_EXP_landmarks.nii.gz")
        save_landmarks(v["target_list"]/np.array((2,2,2)), dist(v["target_list"], v["warped_list"])/2, (175, 175, 175), f"{plot_root}/{k[:-4]}_highres/{k[:-4]}_highres_EXP_landmarks_error.nii.gz")
        save_landmarks(v["warped_list"]/np.array((2,2,2)), np.array(range(v["target_list"].shape[0])), (175, 175, 175), f"{plot_root}/{k[:-4]}_highres/{k[:-4]}_highres_warped_INSP_landmarks.nii.gz")
    else:
        if not os.path.exists(f"{plot_root}/{k}_highres"):
            os.mkdir(f"{plot_root}/{k}_highres")
        save_landmarks(v["target_list"]/np.array((2,2,2)), np.array(range(v["target_list"].shape[0])), (175, 175, 175), f"{plot_root}/{k}_highres/{k}_highres_INSP_landmarks.nii.gz")
        save_landmarks(v["target_list"]/np.array((2,2,2)), dist(v["target_list"], v["warped_list"])/2, (175, 175, 175), f"{plot_root}/{k}_highres/{k}_highres_INSP_landmarks_error.nii.gz")
        save_landmarks(v["warped_list"]/np.array((2,2,2)), np.array(range(v["target_list"].shape[0])),(175, 175, 175), f"{plot_root}/{k}_highres/{k}_highres_warped_EXP_landmarks.nii.gz")

## Plot landmarks

In [61]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

i = 9
def plot(landmarks, errors, cPos):
    return go.Scatter3d(x=landmarks[:,0], y=landmarks[:,1], z=landmarks[:,2], 
                        mode='markers',
                        marker=dict(
                            size=4,
                            color=errors,
                            colorbar=dict(title="Colorbar", x=cPos),
                            cmin=0, cmax=20)
                        )

case_id = cases[i][:-8]
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}]])
fig.add_trace(plot(res[case_id]["target_list"], dist(res[case_id]["target_list"], res[case_id]["warped_list"]), 0), row=1, col=1)
fig.add_trace(plot(res[f"{case_id}_inv"]["target_list"], dist(res[f"{case_id}_inv"]["target_list"], res[f"{case_id}_inv"]["warped_list"]), 0.5), row=1, col=2)
fig.show()

In [62]:
fig = make_subplots(rows=5, cols=2)
for i in range(len(cases)):
    case_id = f"{cases[i][:-8]}"
    fig.append_trace(go.Histogram(x=dist(res[case_id]["target_list"], res[case_id]["warped_list"]), name=case_id,
                xbins={"start":0, "end":35, "size":1}), int(i/2)+1, i%2+1)
fig.show()

# Experiments record

mTRE: 9.72256080624498, mTRE_X: 2.5588051263802747, mTRE_Y: 5.536045490405056, mTRE_Z: 6.067062707815259, DICE: 0.9310127314875011
mTRE: 7.913783452204624, mTRE_X: 2.3584838741721397, mTRE_Y: 4.577090343265437, mTRE_Z: 4.710559147728482, DICE: 0.9534731271322239

mTRE: 12.148867655383487, mTRE_X: 3.3734660434169044, mTRE_Y: 5.8329253136761405, mTRE_Z: 8.503192662817732, DICE: 0.9302245014394808
mTRE: 9.475289404788844, mTRE_X: 3.0082783361810974, mTRE_Y: 5.604369655896521, mTRE_Z: 5.340543340720618, DICE: 0.9400378985755597

mTRE: 7.392256446730748, mTRE_X: 2.193760589986982, mTRE_Y: 4.2038040909034615, mTRE_Z: 4.464984625209381, DICE: 0.93134730795532
mTRE: 6.1353438889191265, mTRE_X: 1.9059608627314197, mTRE_Y: 3.428824476952798, mTRE_Z: 3.7382983575747595, DICE: 0.9530403166574072

mTRE: 15.396210869104243, mTRE_X: 4.831591407918003, mTRE_Y: 7.415095285699425, mTRE_Z: 10.232413528681425, DICE: 0.8379295864286507
mTRE: 11.734486756224474, mTRE_X: 4.544151488982049, mTRE_Y: 6.565068406101193, mTRE_Z: 6.459538645805556, DICE: 0.8548576830336945

mTRE: 11.420655359769952, mTRE_X: 3.2665643854050748, mTRE_Y: 6.417862104649079, mTRE_Z: 7.03283131172154, DICE: 0.919489719908476
mTRE: 9.655326069146609, mTRE_X: 2.9966906256532058, mTRE_Y: 5.626698389500779, mTRE_Z: 5.591218712830669, DICE: 0.9275508900041274

mTRE: 9.84794076582552, mTRE_X: 2.6118545206208497, mTRE_Y: 5.767233874200109, mTRE_Z: 6.011357885688524, DICE: 0.9348090155763437
mTRE: 8.285666507799078, mTRE_X: 2.4718957687728125, mTRE_Y: 4.831986275185797, mTRE_Z: 4.924829635087673, DICE: 0.948274132139489

mTRE: 7.7782961829387585, mTRE_X: 2.2341650306456957, mTRE_Y: 4.383443170494663, mTRE_Z: 4.759806571622792, DICE: 0.9696728373690704
mTRE: 6.763074679956818, mTRE_X: 1.9707038314568532, mTRE_Y: 4.1612984913263995, mTRE_Z: 3.843222421705044, DICE: 0.9751690254210503

mTRE: 8.369776881374776, mTRE_X: 2.184151748467625, mTRE_Y: 5.081952402948656, mTRE_Z: 4.882359755425627, DICE: 0.9663634068288983
mTRE: 6.72805719342386, mTRE_X: 2.0122995478869146, mTRE_Y: 3.8373160766690426, mTRE_Z: 4.092259168506311, DICE: 0.9738081114557369

down8x
phi:esp->insp: mTRE: 11.647122472445767, mTRE_X: 3.3054521429618497, mTRE_Y: 5.743549408170697, mTRE_Z: 7.932908809029682, DICE: 0.8719203405071122
phi:insp->esp: mTRE: 9.058389898332077, mTRE_X: 2.8747980570077027, mTRE_Y: 4.524412303294483, mTRE_Z: 5.883520439586486, DICE: 0.8800464274053226

down4x
phi:esp->insp: mTRE: 9.771012291593777, mTRE_X: 2.7346249463863535, mTRE_Y: 5.268773517084363, mTRE_Z: 6.252755623484553, DICE: 0.9304972315170627
phi:insp->esp: mTRE: 8.211809703666468, mTRE_X: 2.4868637222455248, mTRE_Y: 4.428542154229149, mTRE_Z: 5.1437020204719115, DICE: 0.9407695347031128

phi:esp->insp: mTRE: 7.30985730042251, mTRE_X: 2.15082545841491, mTRE_Y: 4.213200104035219, mTRE_Z: 4.3007393386332575, DICE: 0.9285749674672014
phi:insp->esp: mTRE: 6.016069581819211, mTRE_X: 1.808109696297139, mTRE_Y: 3.448185341349853, mTRE_Z: 3.5479899138264637, DICE: 0.9434653988335878

phi:esp->insp: mTRE: 7.016578894729196, mTRE_X: 2.1057356289927927, mTRE_Y: 4.074715115426432, mTRE_Z: 4.086488792337174, DICE: 0.9304695945068232
phi:insp->esp: mTRE: 5.702797890865925, mTRE_X: 1.75123661765845, mTRE_Y: 3.2841157848990696, mTRE_Z: 3.3404162899517296, DICE: 0.9450094134013465

down2x
phi:esp->insp: mTRE: 6.830280447138567, mTRE_X: 2.038814452060917, mTRE_Y: 4.119642250620894, mTRE_Z: 3.826184000006193, DICE: 0.9601948529927856
phi:insp->esp: mTRE: 5.479810645191277, mTRE_X: 1.6529249281942229, mTRE_Y: 3.2290710839462156, mTRE_Z: 3.131063718973693, DICE: 0.9703141290188826

phi:esp->insp: mTRE: 5.835654895803032, mTRE_X: 1.81996563431871, mTRE_Y: 3.4571738117562942, mTRE_Z: 3.2741323729333685, DICE: 0.9704737897027312
phi:insp->esp: mTRE: 4.739657614501623, mTRE_X: 1.4960736049256531, mTRE_Y: 2.7518144379320093, mTRE_Z: 2.736624075530003, DICE: 0.9773067224116077

EndtoEnd
- phi:esp->insp: mTRE: 5.7409646799990774, mTRE_X: 1.6932473874771197, mTRE_Y: 3.4856897765568027, mTRE_Z: 3.2679481404259874, DICE: 0.9726466987376666
- phi:insp->esp: mTRE: 4.63714451145877, mTRE_X: 1.4517570100069783, mTRE_Y: 2.5967982576497812, mTRE_Z: 2.775520829874894, DICE: 0.9799904630829681
- 160 epoch
    - phi:esp->insp: mTRE: 5.445266084411211, mTRE_X: 1.650950532672303, mTRE_Y: 3.3409913813986973, mTRE_Z: 3.004634778710078, DICE: 0.9695773180296576
    - phi:insp->esp: mTRE: 4.189016530859336, mTRE_X: 1.3796167169346876, mTRE_Y: 2.3983895223948113, mTRE_Z: 2.4190064973981498, DICE: 0.9796126422971362
- 200 epoch
    - phi:esp->insp: mTRE: 5.965608239455672, mTRE_X: 1.7605315385021747, mTRE_Y: 3.588892192470749, mTRE_Z: 3.400352378086955, DICE: 0.973110779699988
    - phi:insp->esp: mTRE: 4.663346517668652, mTRE_X: 1.4856102224932137, mTRE_Y: 2.604851837841072, mTRE_Z: 2.757780305980546, DICE: 0.9807705826008529
- 220 epoch
    - phi:esp->insp: mTRE: 6.276826269354277, mTRE_X: 1.8575498121008895, mTRE_Y: 3.7438447885654518, mTRE_Z: 3.6288783905201556, DICE: 0.9724593347798145
    - phi:insp->esp: mTRE: 4.784226391727859, mTRE_X: 1.6063437963403284, mTRE_Y: 2.721001699840217, mTRE_Z: 2.7472474079440232, DICE: 0.9814962326077188

EndtoEnd scale to [0,1]
- epoch 120
    - phi:esp->insp: mTRE: 4.366484556291117, mTRE_X: 1.4251501745842776, mTRE_Y: 2.5797625928912473, mTRE_Z: 2.406453559417143, DICE: 0.9728284807100227
    - phi:insp->esp: mTRE: 3.9503488290795197, mTRE_X: 1.273623756376107, mTRE_Y: 2.2158012972015304, mTRE_Z: 2.310259530581153, DICE: 0.9773167429536201
- epoch 140
    - phi:esp->insp: mTRE: 4.3809571203654585, mTRE_X: 1.4034924517240115, mTRE_Y: 2.6753403718914983, mTRE_Z: 2.3495489330840034, DICE: 0.9724784692885065
    - phi:insp->esp: mTRE: 3.9184406837331096, mTRE_X: 1.3227945364432636, mTRE_Y: 2.1582292123549576, mTRE_Z: 2.2932822704126443, DICE: 0.9774326006867817
- epoch 200
    - phi:esp->insp: mTRE: 4.281145751643484, mTRE_X: 1.34408576404081, mTRE_Y: 2.5586140470270187, mTRE_Z: 2.3553608005477384, DICE: 0.9737925566085345
    - phi:insp->esp: mTRE: 3.870456789762393, mTRE_X: 1.282384988184878, mTRE_Y: 2.1307803707272557, mTRE_Z: 2.288719105339985, DICE: 0.977155799897738